# What sequence to watch? X-Men edition.

#https://www.denofgeek.com/uk/movies/x-men-movies/48242/x-men-best-order-to-watch-the-movies-in 

* has released various lists on how to watch X-Men Movies. Since these movies jump time lines, various people prefer different viewing approach.
* But what if ***someone only wants to know the sequence IN-BEWEEN? or after certain movies watched already?***

Q-Learning provides random yet correct sequences to the user based on his ***start*** and ***end*** movie specification. 

In [1]:
import numpy as np

* According to ***Den of Geek*** There are 8 possible ways one can watch the series.
* They are (according to how I have placed them in the single list below):
<ol>
    <li> In production order </li>
    <li> Without Deadpool series (My addition for more data) </li>
    <li> In chronological order </li>
    <li> Days of Future past future setting </li>
    <li> Internal Chronology </li>
    <li> New Timeline </li>
    <li> Wolverine's Story </li>
    <li> Charles Xavier's Story </li>
    <li> "The happy ending" viewing order </li>
    
<ol>
<p> Data format </p>
    <li> evert sequence has to be a single long string separated by commas </li>
    <li> These strings make up the list. </li>

In [222]:
x_men_order = ['xmen,xmen2,xmen_last_stand,xmen_origins_wolverine,xmen_first_class,the_wolverine,x_men_days_of_future_past,deadpool,x_men_apocalypse,logan,deadpool2,xmen_dark_pheonix',
              'xmen,xmen2,xmen_last_stand,xmen_origins_wolverine,xmen_first_class,the_wolverine,x_men_days_of_future_past,x_men_apocalypse,logan,xmen_dark_pheonix',
              'xmen_first_class,x_men_days_of_future_past,xmen_origins_wolverine,x_men_apocalypse,xmen_dark_pheonix,xmen,xmen2,xmen_last_stand,the_wolverine,deadpool,deadpool2',
              'xmen_first_class,xmen_origins_wolverine,x_men_apocalypse,xmen_dark_pheonix,xmen,xmen2,xmen_last_stand,the_wolverine,deadpool,deadpool2,x_men_days_of_future_past,logan',
              'xmen_first_class,xmen_origins_wolverine,xmen,xmen2,xmen_last_stand,the_wolverine,x_men_days_of_future_past',
              'xmen_first_class,x_men_days_of_future_past,x_men_apocalypse,xmen_dark_pheonix,deadpool,deadpool2,logan',
              'xmen_origins_wolverine,xmen_first_class,x_men_apocalypse,xmen,xmen2,xmen_last_stand,the_wolverine,deadpool,x_men_days_of_future_past,logan',
              'xmen_first_class,x_men_days_of_future_past,xmen_origins_wolverine,x_men_apocalypse,xmen_dark_pheonix,xmen,xmen2,xmen_last_stand,the_wolverine,deadpool,logan',
              'xmen_first_class,xmen_origins_wolverine,x_men_days_of_future_past,x_men_apocalypse,xmen_dark_pheonix,xmen,xmen2,xmen_last_stand,the_wolverine,logan,deadpool,deadpool2']

Lets tokenize our movies into numbers. 
* Location_to_state has word->token mapping.
* State_to_location has token->word mapping.

In [223]:
def data_dicts(X_):
    word_to_ix={}
    ix_to_word={}
    x=0
    for partial_quesries in X_:
        partial_quesries = partial_quesries.split(",")
        for word in partial_quesries:
            if word not in word_to_ix.keys():
                word_to_ix[word]=x
                ix_to_word[x]=word
                x+=1
    return (word_to_ix,ix_to_word)
location_to_state,state_to_location=data_dicts(x_men_order)    
# location_to_state1,state_to_location1=data_dicts(sequence_list)

Lets have a tokenized version of our main list.

In [224]:
def list_to_num_seq(x, loc_to_state):
    token_seq = []
    for i in x:
        i = i.split(",")
        token_intermediate = []
        for j in i:
            token_intermediate.append(loc_to_state[j])
        token_seq.append(token_intermediate)
    return token_seq

In [225]:
path_array = np.array(list_to_num_seq(x_men_order, location_to_state))

In [226]:
path_array

array([list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
       list([0, 1, 2, 3, 4, 5, 6, 8, 9, 11]),
       list([4, 6, 3, 8, 11, 0, 1, 2, 5, 7, 10]),
       list([4, 3, 8, 11, 0, 1, 2, 5, 7, 10, 6, 9]),
       list([4, 3, 0, 1, 2, 5, 6]), list([4, 6, 8, 11, 7, 10, 9]),
       list([3, 4, 8, 0, 1, 2, 5, 7, 6, 9]),
       list([4, 6, 3, 8, 11, 0, 1, 2, 5, 7, 9]),
       list([4, 3, 6, 8, 11, 0, 1, 2, 5, 9, 7, 10])], dtype=object)

Lets have a discount factor and learning rate for our Q-Learning model.

*Alpha is the learning rate. If the reward or transition function is stochastic (random), then alpha should change over time, approaching zero at infinity. This has to do with approximating the expected outcome of a inner product (T(transition)*R(reward)), when one of the two, or both, have random behavior.

*Gamma is the value of future reward. It can affect learning quite a bit, and can be a dynamic or static value. If it is equal to one, the agent values future reward JUST AS MUCH as current reward. This means, in ten actions, if an agent does something good this is JUST AS VALUABLE as doing this action directly. So learning doesn't work at that well at high gamma values.

*Explanation above thanks to: https://stackoverflow.com/a/18543274/12752884

In [227]:
gamma = 0.75 # Discount factor 
alpha = 0.9 # Learning rate

We would need a rewards matrix. Hardcoding is a pain, lets write a function!

In [228]:
def gen_reward_matrix(a_array,max_elem):
    t= np.zeros((max_elem+3,max_elem+3))
    di = dict()
    for i in range(0,len(a_array)-1):
        if a_array[i] in di.keys():
            di[a_array[i]].append(a_array[i+1])
        else:
            di[a_array[i]] = a_array[i+1]
    if a_array[-1] not in di:
        di[a_array[-1]]=a_array[-1]+1
    else:
        di[a_array[-1]].append(a_array[-2])
    
    for i in range(max_elem+3):
        if i in di.keys():
            ls = di[i] 
            t[i][di[i]]=1
    return t

* Our model is a granular level model, meaning it does not create reward matrix for the entire corpus, but only one sequence at a time. 
* We randomly pick a sequence which has both the start and the end nodes. 
* Yes, this will not make sense if the the start and end nodes are the very start and end nodes of a sequence, but what if the start node or the end node lies between? That is, what if a user wants to watch from a certain movie, not necessarily the beginning of any order?

In [229]:
def obtain_sequence_pattern(a_arr,start_location,end_location,location_to_state):
    for i in a_arr:
        if (location_to_state[start_location] in i) and (location_to_state[end_location] in i):
            if i.index(location_to_state[start_location])<i.index(location_to_state[end_location]):
                a = i
                break
    return a
    

The main Q-Learning function ***gen_sequence*** returns us the main sequence as it utilizes both the ***gen_reward_matrix*** and the ***obtain_sequence_pattern*** functions. 

In [230]:
import random
def gen_sequence(start_location,end_location,location_to_state,state_to_location, a_arr):
    a_arr = sorted(a_arr, key=lambda k: random.random())
    a_array = obtain_sequence_pattern(a_arr,start_location,end_location,location_to_state)
    max_elem = max(a_array)
    rewards = gen_reward_matrix(a_array,max(a_array))
    rewards_new = np.copy(rewards)
    ending_state = location_to_state[end_location]
    rewards_new[ending_state,ending_state] = 999
    Q = np.array(np.zeros([max_elem+1,max_elem+1]))

    for i in range(1000):
        current_state = np.random.randint(0,max_elem+1) # Python excludes the upper bound
        playable_actions = []
        for j in range(max_elem+1):
            if rewards_new[current_state,j] > 0:
                playable_actions.append(j)
        if playable_actions:
            next_state = np.random.choice(playable_actions)
            TD = rewards_new[current_state,next_state] + gamma * Q[next_state, np.argmax(Q[next_state,])] - Q[current_state,next_state]
            Q[current_state,next_state] += alpha * TD
  
    route = [start_location]
    
    next_location = start_location
    
    while(next_location != end_location):
        starting_state = location_to_state[start_location]    
        
        next_state = np.argmax(Q[starting_state,])
    
        next_location = state_to_location[next_state]
        route.append(next_location)
        start_location = next_location
    return route

Lets check, what if we want to start from the movie ***x-men*** and watch until ***x-men dark pheonix***?  

In [231]:

gen_sequence('xmen','xmen_dark_pheonix',location_to_state,state_to_location,path_array)

['xmen',
 'xmen2',
 'xmen_last_stand',
 'xmen_origins_wolverine',
 'xmen_first_class',
 'the_wolverine',
 'x_men_days_of_future_past',
 'x_men_apocalypse',
 'logan',
 'xmen_dark_pheonix']

Lets generate all the ***valid*** possible sequences one can watch if the required start point is ***X-Men First Class*** and end point is ***Logan***?

In [232]:
result_lst = []
for i in range(10):
    result_lst.append(gen_sequence('xmen_first_class','logan',location_to_state,state_to_location,path_array))
    
# print(result_lst)
seq = [list(item) for item in set(tuple(row) for row in result_lst)]
seq

[['xmen_first_class',
  'x_men_days_of_future_past',
  'xmen_origins_wolverine',
  'x_men_apocalypse',
  'xmen_dark_pheonix',
  'xmen',
  'xmen2',
  'xmen_last_stand',
  'the_wolverine',
  'deadpool',
  'logan'],
 ['xmen_first_class',
  'xmen_origins_wolverine',
  'x_men_days_of_future_past',
  'x_men_apocalypse',
  'xmen_dark_pheonix',
  'xmen',
  'xmen2',
  'xmen_last_stand',
  'the_wolverine',
  'logan'],
 ['xmen_first_class',
  'the_wolverine',
  'x_men_days_of_future_past',
  'deadpool',
  'x_men_apocalypse',
  'logan'],
 ['xmen_first_class',
  'x_men_days_of_future_past',
  'x_men_apocalypse',
  'xmen_dark_pheonix',
  'deadpool',
  'deadpool2',
  'logan'],
 ['xmen_first_class',
  'x_men_apocalypse',
  'xmen',
  'xmen2',
  'xmen_last_stand',
  'the_wolverine',
  'deadpool',
  'x_men_days_of_future_past',
  'logan']]

In [237]:
state_to_location

{0: 'xmen',
 1: 'xmen2',
 2: 'xmen_last_stand',
 3: 'xmen_origins_wolverine',
 4: 'xmen_first_class',
 5: 'the_wolverine',
 6: 'x_men_days_of_future_past',
 7: 'deadpool',
 8: 'x_men_apocalypse',
 9: 'logan',
 10: 'deadpool2',
 11: 'xmen_dark_pheonix'}

Lets randomly create a start point and end point to see some of the sequences. (May not always work if the main list doesn't have both the start and end points in any of the sublists.

* One can choose to watch Deadpool and Deadpool 2 together ***or*** Deadpool -> X-Men Apocalypse -> Logan -> Deadpool 2.

In [347]:
random_start = np.random.randint(0, 11)
if random_start!=11:
    random_end = np.random.randint(random_start+1, 11)
else:
    random_start = np.random.randint(0, 11)
    random_end = np.random.randint(random_start+1, 11)
print("random_seeds:", random_start, random_end)
print("start movie:", state_to_location[random_start],"\nend movie:",state_to_location[random_end])

result_lst = []
for i in range(10):
    result_lst.append(gen_sequence(state_to_location[random_start],state_to_location[random_end],location_to_state,state_to_location,path_array))
    
# print(result_lst)
seq = [list(item) for item in set(tuple(row) for row in result_lst)]
seq

random_seeds: 7 10
start movie: deadpool 
end movie: deadpool2


[['deadpool', 'deadpool2'],
 ['deadpool', 'x_men_apocalypse', 'logan', 'deadpool2']]

To end this, lets generate all the possible sequences.

In [211]:
def get_unique_seq(alist):
    x = []
    for i in alist:
        i = i.split(",")
        x.append([i[0],i[-1]])
    x = [list(item) for item in set(tuple(row) for row in x)]
    return x

In [212]:
def obtain_all_sequences():    
    uniq = get_unique_seq(x_men_order)
    seq_i = []
    for i in uniq:
        print(i)
        result_lst = []
        for j in range(50):
            result_lst.append(gen_sequence(i[0],i[-1],location_to_state,state_to_location,path_array))
        only_unique = [list(item) for item in set(tuple(row) for row in result_lst)]
        seq_i.append(only_unique)
    return seq_i

In [213]:
seq_i = obtain_all_sequences()

['xmen_first_class', 'logan']
['xmen_first_class', 'deadpool2']
['xmen_first_class', 'x_men_days_of_future_past']
['xmen', 'xmen_dark_pheonix']
['xmen_origins_wolverine', 'logan']


All the possible sequences generated using our model. (Only for the purpose of testing).

In [215]:
seq_i

[[['xmen_first_class',
   'xmen_origins_wolverine',
   'x_men_apocalypse',
   'xmen_dark_pheonix',
   'xmen',
   'xmen2',
   'xmen_last_stand',
   'the_wolverine',
   'deadpool',
   'deadpool2',
   'x_men_days_of_future_past',
   'logan'],
  ['xmen_first_class',
   'x_men_days_of_future_past',
   'xmen_origins_wolverine',
   'x_men_apocalypse',
   'xmen_dark_pheonix',
   'xmen',
   'xmen2',
   'xmen_last_stand',
   'the_wolverine',
   'deadpool',
   'logan'],
  ['xmen_first_class',
   'xmen_origins_wolverine',
   'x_men_days_of_future_past',
   'x_men_apocalypse',
   'xmen_dark_pheonix',
   'xmen',
   'xmen2',
   'xmen_last_stand',
   'the_wolverine',
   'logan'],
  ['xmen_first_class',
   'x_men_days_of_future_past',
   'x_men_apocalypse',
   'xmen_dark_pheonix',
   'deadpool',
   'deadpool2',
   'logan'],
  ['xmen_first_class',
   'x_men_apocalypse',
   'xmen',
   'xmen2',
   'xmen_last_stand',
   'the_wolverine',
   'deadpool',
   'x_men_days_of_future_past',
   'logan']],
 [['xmen_